# Dogs vs. Cats Redux: Kernels Edition

## 2. 모델구성 및 학습

In [1]:
import os
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
import time
import re

# 구현모듈
import data
import model
import train

/home/shkim/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### 2.1 학습 데이터 생성 함수

In [2]:
def getFiles(dir, word):
    files =   [dir+i for i in os.listdir(dir) if word in i]
    files.sort(key=lambda var:[int(x) if x.isdigit() else x for x in re.findall(r'[^0-9]|[0-9]+', var)])
    return files

In [3]:
def getDogCatData(image_dir, test_size=0.3):
    dog_files =  getFiles(image_dir, 'dog')
    cat_files =  getFiles(image_dir, 'cat')
    print("dog_files: ", len(dog_files))
    print("cat_files: ", len(cat_files))
    
    #dog_files = dog_files[:200]
    #cat_files = cat_files[:200]

    dog_cat_files = dog_files + cat_files

    label_dog = [0 for i in range(len(dog_files))]
    label_cat = [1 for i in range(len(cat_files))]
    label_one_hot = np.eye(2)[label_dog + label_cat]

    return train_test_split (dog_cat_files, label_one_hot, test_size=test_size, random_state=42)

### 2.2 학습데이터와 valid 데이터 분리

In [4]:
DATA_DIR = "../data/"
TRAIN_DIR = DATA_DIR + "train_resize/"

x_train, x_valid, y_train, y_valid = getDogCatData (TRAIN_DIR)

print("train: ", len(x_train))
print("valid: ", len(x_valid))

dog_files:  12500
cat_files:  12500
train:  17500
valid:  7500


In [5]:
learning_rate = 0.0001
epochs = 50
batch_size =16

### 2.3 dataset 생성

In [6]:
# dataset 생성
train_dataset = data.Dataset_image([x_train, y_train], batch_size = batch_size)
valid_dataset = data.Dataset_image([x_valid, y_valid], batch_size = batch_size)

In [7]:
sess = tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth =True)))
model = model.Vgg(sess, "model")
model.build_net(image_shape=[150, 150, 3], class_count=2)

Instructions for updating:
Use the retry module or similar alternatives.


### 2.4 학습

In [8]:
train.train (model, train_dataset, valid_dataset, learning_rate, epochs)

train_count:  17500 total_batch_train:  1094
valid_count:  7500 total_batch_valid:  469
Learning started. It takes sometime.
Epoch: 0001 train [cost:  1.304292083 , acc: 0.5146] valid [cost:  0.942297918 , acc: 0.5046]  749.73 seconds
Epoch: 0002 train [cost:  1.009880403 , acc: 0.5265] valid [cost:  1.378580610 , acc: 0.5033]  745.77 seconds
Epoch: 0003 train [cost:  0.960882443 , acc: 0.5307] valid [cost:  1.583931538 , acc: 0.5040]  745.99 seconds
Epoch: 0004 train [cost:  0.916774880 , acc: 0.5534] valid [cost:  1.245467574 , acc: 0.5141]  745.86 seconds
Epoch: 0005 train [cost:  0.863995445 , acc: 0.5993] valid [cost:  0.869077190 , acc: 0.5615]  745.85 seconds
Epoch: 0006 train [cost:  0.834068348 , acc: 0.6350] valid [cost:  0.681089581 , acc: 0.6525]  745.77 seconds
Epoch: 0007 train [cost:  0.783637080 , acc: 0.6680] valid [cost:  1.179714743 , acc: 0.6169]  745.87 seconds
Epoch: 0008 train [cost:  0.760678464 , acc: 0.6910] valid [cost:  0.908118124 , acc: 0.6613]  746.19 sec

### 2.5 평가

In [9]:
train.eval (model, train_dataset)

eval_count:  17500 , total_batch_eval:  1094
Evaluation Finished!
cost:  0.153674873 , accuracy: 0.9542
--- 151.87 seconds ---


(0.15367487348412653, 0.9541628580315626)

In [10]:
train.eval (model, valid_dataset)

eval_count:  7500 , total_batch_eval:  469
Evaluation Finished!
cost:  0.487917748 , accuracy: 0.8917
--- 65.11 seconds ---


(0.48791774778131897, 0.8917466240651077)